# YouTube Report Generator - Part 1: Setup & Preprocessing

This notebook contains environment setup, configuration, and preprocessing utilities.

**Pipeline Structure:**
- **Part 1** (this file): Setup, configuration, utilities, data loading
- **Part 2**: Category and Sentiment model initialization
- **Part 3**: Full pipeline execution and report generation

Run all cells in order before proceeding to Part 2.

## 1. Environment Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [2]:
!pip install -q transformers accelerate bitsandbytes torch
!pip install -q langdetect isodate
!pip install -q google-api-python-client
!pip install -q imbalanced-learn datasets seaborn scikit-learn scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import json
import re
import os
import gc
import warnings
from datetime import datetime
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field

import torch
import numpy as np
import isodate
from langdetect import detect, LangDetectException
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA device: NVIDIA L4


## 2. Configuration

In [4]:
@dataclass
class PipelineConfig:
    """Full pipeline configuration"""

    # Model settings
    model_name: str = "meta-llama/Llama-3.1-8B-Instruct"
    use_4bit: bool = True
    max_new_tokens: int = 512
    temperature: float = 0.7
    top_p: float = 0.9

    # Data settings
    use_youtube_api: bool = False
    youtube_api_key: Optional[str] = None
    data_path: str = "/content/drive/MyDrive/25_sch/ML/youtube_test_10132154/full_dataset_20251013_215347.json"

    # Processing settings
    max_description_length: int = 2000
    max_comments_to_process: int = 100
    min_comment_length: int = 10
    remove_urls: bool = True
    detect_language: bool = True

    # Output settings
    output_language: str = "English"
    multilingual_understanding: bool = True

    # Test settings
    num_videos_for_test: int = 3
    enable_detailed_logging: bool = True
    log_token_counts: bool = True

    # Token efficiency settings
    enable_dynamic_tokens: bool = True
    token_efficiency_mode: str = "adaptive"  # 'adaptive', 'conservative', 'aggressive'

    # Quality gate settings
    enable_quality_gate: bool = True
    min_summary_length: int = 100
    min_keyword_diversity: int = 10
    max_regeneration_attempts: int = 2
    quality_gate_temperature: float = 0.5

    # Category model settings
    use_category_model: bool = True
    category_model_path: Optional[str] = "/content/drive/MyDrive/25_sch/ML/models"
    enable_llm_category_correction: bool = True

    # Sentiment model settings
    use_sentiment_model: bool = True
    sentiment_root_dir: str = "./models/sentiment"
    sentiment_model_path: str = "./models/sentiment/bert_sentiment"
    sentiment_zip_path: str = "/content/drive/MyDrive/25_sch/ML/sentiment/bert_sentiment.zip"

    def __post_init__(self):
        if self.use_youtube_api and not self.youtube_api_key:
            raise ValueError("YouTube API key is required when use_youtube_api=True")


# Create global config instance
config = PipelineConfig()

print("=" * 60)
print("Pipeline Configuration")
print("=" * 60)
print(f"Model: {config.model_name}")
print(f"4-bit quantization: {config.use_4bit}")
print(f"Output Language: {config.output_language}")
print(f"Test Videos: {config.num_videos_for_test}")
print(f"Max Comments: {config.max_comments_to_process}")
print(f"Token Efficiency: {config.enable_dynamic_tokens} (Mode: {config.token_efficiency_mode})")
print(f"Quality Gate: {config.enable_quality_gate}")
print(f"Category Model: {config.use_category_model}")
print(f"Sentiment Model: {config.use_sentiment_model}")
print("=" * 60)

Pipeline Configuration
Model: meta-llama/Llama-3.1-8B-Instruct
4-bit quantization: True
Output Language: English
Test Videos: 3
Max Comments: 100
Token Efficiency: True (Mode: adaptive)
Quality Gate: True
Category Model: True
Sentiment Model: True


## 3. Prompt Templates

In [5]:
class PromptTemplates:
    """Prompt templates for video analysis"""

    @staticmethod
    def get_video_summary_prompt(title: str, description: str, category_id: str,
                                  duration: str, output_lang: str = "English") -> str:
        """Video summary prompt with two-part structure"""
        return f"""You are analyzing a YouTube video. Based on the information provided, create a comprehensive summary in {output_lang}.

Video Information:
- Title: {title}
- Description: {description}
- Category ID: {category_id}
- Duration: {duration}

Instructions:
1. First, provide a KEY POINT (1-2 sentences) that captures the video's essence
2. Then, provide a DETAILED SUMMARY (3-5 sentences) with more context
3. Write ONLY in {output_lang}, regardless of the input language
4. Focus on what viewers will learn or experience
5. Be concise and informative

Format your response EXACTLY as:
KEY POINT:
[Your 1-2 sentence key point here]

DETAILED SUMMARY:
[Your 3-5 sentence detailed summary here]
"""

    @staticmethod
    def get_reaction_summary_prompt(comments: List[str], output_lang: str = "English") -> str:
        """Reaction summary prompt with sentiment analysis"""
        comments_text = "\n".join([f"- {c}" for c in comments[:100]])
        return f"""You are analyzing audience reactions to a YouTube video. Based on the comments provided, create a comprehensive reaction summary in {output_lang}.

Comments:
{comments_text}

Instructions:
1. First, provide a KEY POINT (1-2 sentences) about the overall reaction
2. Then, provide a DETAILED SUMMARY (3-5 sentences) with:
   - Majority opinion (what most viewers think)
   - Notable minority views (outliers, different perspectives)
   - Language patterns (if mixed Korean/English/Japanese, note code-switching)
3. After the summary, provide a SENTIMENT BREAKDOWN estimate:
   Positive: X%  Negative: Y%  Neutral: Z%
4. Write ONLY in {output_lang}, regardless of comment languages
5. Ignore spam, absurd, or irrelevant comments (<5% outliers)

Format your response EXACTLY as:
KEY POINT:
[Your 1-2 sentence key point here]

DETAILED SUMMARY:
[Your 3-5 sentence detailed analysis here]

SENTIMENT BREAKDOWN:
Positive: X%  Negative: Y%  Neutral: Z%
"""

    @staticmethod
    def get_category_correction_prompt(title: str, description: str,
                                       predicted_category: int) -> str:
        """Category validation prompt"""
        return f"""You are reviewing a video category classification.

Video Information:
- Title: {title}
- Description: {description[:500]}
- Model Predicted Category: {predicted_category}

Valid Categories:
1: Film & Animation, 2: Autos & Vehicles, 10: Music, 15: Pets & Animals,
17: Sports, 19: Travel & Events, 20: Gaming, 22: People & Blogs,
23: Comedy, 24: Entertainment, 25: News & Politics, 26: HowTo & Style,
27: Education, 28: Science & Tech

Task: Review if the predicted category is appropriate. Respond with ONLY:
- "CORRECT" if the prediction is appropriate
- "CORRECT: [category_id]" if you suggest a different category (use the number)

Example responses:
- "CORRECT" (if prediction is good)
- "CORRECT: 10" (if Music is more appropriate)
"""

print("Prompt templates loaded")

Prompt templates loaded


## 4. Text Preprocessing Utilities

In [6]:
class TextPreprocessor:
    """Text preprocessing utilities"""

    # Language name mapping
    LANG_NAMES = {
        'ko': 'Korean', 'en': 'English', 'ja': 'Japanese',
        'zh-cn': 'Chinese (Simplified)', 'zh-tw': 'Chinese (Traditional)',
        'de': 'German', 'fr': 'French', 'es': 'Spanish',
        'it': 'Italian', 'pt': 'Portuguese', 'ru': 'Russian',
        'ar': 'Arabic', 'hi': 'Hindi', 'th': 'Thai', 'vi': 'Vietnamese',
        'nl': 'Dutch', 'pl': 'Polish', 'tr': 'Turkish',
        'sv': 'Swedish', 'da': 'Danish', 'no': 'Norwegian', 'fi': 'Finnish',
        'cs': 'Czech', 'hu': 'Hungarian', 'ro': 'Romanian', 'uk': 'Ukrainian',
        'el': 'Greek', 'he': 'Hebrew', 'af': 'Afrikaans', 'ca': 'Catalan',
        'hr': 'Croatian', 'sr': 'Serbian', 'sk': 'Slovak', 'sl': 'Slovenian',
        'et': 'Estonian', 'lv': 'Latvian', 'lt': 'Lithuanian', 'bg': 'Bulgarian',
        'fa': 'Persian', 'ur': 'Urdu', 'bn': 'Bengali', 'ta': 'Tamil',
        'te': 'Telugu', 'mr': 'Marathi', 'kn': 'Kannada',
        'ml': 'Malayalam', 'gu': 'Gujarati'
    }

    @staticmethod
    def remove_urls(text: str) -> str:
        """Remove URLs from text"""
        url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        return re.sub(url_pattern, '', text)

    @staticmethod
    def clean_text(text: str, remove_urls: bool = True) -> str:
        """Clean text by removing URLs and extra whitespace"""
        if not text:
            return ""
        if remove_urls:
            text = TextPreprocessor.remove_urls(text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    @staticmethod
    def detect_language(text: str) -> str:
        """Detect text language and return full name"""
        try:
            if not text or len(text.strip()) < 10:
                return "Unknown"
            lang = detect(text)
            return TextPreprocessor.LANG_NAMES.get(lang, lang.title())
        except LangDetectException:
            return "Unknown"

    @staticmethod
    def get_language_distribution(texts: List[str]) -> Dict[str, int]:
        """Calculate language distribution for a list of texts"""
        lang_dist = {}
        for text in texts:
            lang = TextPreprocessor.detect_language(text)
            lang_dist[lang] = lang_dist.get(lang, 0) + 1
        return lang_dist

print("Text preprocessing utilities loaded")

Text preprocessing utilities loaded


## 5. Model Manager

In [7]:
class ModelManager:
    """LLM model manager with token efficiency tracking"""

    def __init__(self, config: PipelineConfig):
        self.config = config
        self.model = None
        self.tokenizer = None
        self.pipe = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.token_stats = {
            "total_input_tokens": 0,
            "total_output_tokens": 0,
            "total_calls": 0,
            "tokens_saved": 0
        }

    def load_model(self):
        """Load model with optional 4-bit quantization"""
        print(f"Loading {self.config.model_name}...")

        self.tokenizer = AutoTokenizer.from_pretrained(
            self.config.model_name,
            trust_remote_code=True
        )
        self.tokenizer.pad_token = self.tokenizer.eos_token

        if self.config.use_4bit:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True
            )
            self.model = AutoModelForCausalLM.from_pretrained(
                self.config.model_name,
                quantization_config=bnb_config,
                device_map="auto",
                trust_remote_code=True
            )
        else:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.config.model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )

        self.pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device_map="auto"
        )

        print(f"Model loaded on {self.device}")
        print(f"Memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

    def dynamic_max_tokens(self, input_length: int) -> int:
        """Calculate dynamic token limit based on input length"""
        if not self.config.enable_dynamic_tokens:
            return self.config.max_new_tokens

        mode = self.config.token_efficiency_mode
        if mode == "conservative":
            if input_length < 500:
                return 320
            elif input_length < 1500:
                return 448
            return 512
        elif mode == "aggressive":
            if input_length < 500:
                return 192
            elif input_length < 1500:
                return 320
            return 448
        else:  # adaptive (default)
            if input_length < 500:
                return 256
            elif input_length < 1500:
                return 384
            return 512

    def generate(self, prompt: str, temperature: Optional[float] = None,
                 force_max_tokens: Optional[int] = None) -> str:
        """Generate text with dynamic token allocation"""
        if self.pipe is None:
            raise RuntimeError("Model not loaded. Call load_model() first.")

        input_tokens = len(self.tokenizer.encode(prompt))
        max_tokens = force_max_tokens if force_max_tokens else self.dynamic_max_tokens(input_tokens)
        tokens_saved = self.config.max_new_tokens - max_tokens

        if self.config.enable_dynamic_tokens and tokens_saved > 0:
            self.token_stats["tokens_saved"] += tokens_saved

        if self.config.log_token_counts:
            print(f"      [Input: {input_tokens} tokens, Max output: {max_tokens}]")

        messages = [{"role": "user", "content": prompt}]
        prompt_text = self.tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        temp = temperature if temperature is not None else self.config.temperature
        outputs = self.pipe(
            prompt_text,
            max_new_tokens=max_tokens,
            temperature=temp,
            top_p=self.config.top_p,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )

        response = outputs[0]["generated_text"][len(prompt_text):].strip()
        output_tokens = len(self.tokenizer.encode(response))

        if self.config.log_token_counts:
            print(f"      [Output: {output_tokens} tokens]")

        self.token_stats["total_input_tokens"] += input_tokens
        self.token_stats["total_output_tokens"] += output_tokens
        self.token_stats["total_calls"] += 1

        return response

    def get_token_efficiency_report(self) -> str:
        """Generate token efficiency report"""
        stats = self.token_stats
        total_tokens = stats["total_input_tokens"] + stats["total_output_tokens"]
        avg_per_call = total_tokens / stats["total_calls"] if stats["total_calls"] > 0 else 0
        efficiency = (stats['tokens_saved'] / total_tokens * 100) if total_tokens > 0 else 0

        return f"""
╔══════════════════════════════════════════════════════════╗
║              TOKEN EFFICIENCY REPORT                     ║
╠══════════════════════════════════════════════════════════╣
║ Total Calls:           {stats['total_calls']:>6}                         ║
║ Total Input Tokens:    {stats['total_input_tokens']:>6}                         ║
║ Total Output Tokens:   {stats['total_output_tokens']:>6}                         ║
║ Tokens Saved:          {stats['tokens_saved']:>6}                         ║
║ Avg Tokens/Call:       {avg_per_call:>6.1f}                         ║
║ Efficiency Gain:       {efficiency:>5.1f}%                         ║
╚══════════════════════════════════════════════════════════╝
"""

print("ModelManager loaded")

ModelManager loaded


## 6. Global Model Management

In [8]:
# Global model manager storage
_global_model_manager = None
_global_model_config = None


def get_or_create_model_manager(config: PipelineConfig,
                                 force_reload: bool = False) -> ModelManager:
    """
    Get existing or create new model manager.

    Args:
        config: Pipeline configuration
        force_reload: If True, reload model even if already loaded

    Returns:
        ModelManager instance
    """
    global _global_model_manager, _global_model_config

    if force_reload and _global_model_manager is not None:
        print("Force reload requested...")
        cleanup_model()

    if _global_model_manager is None:
        print(f"Loading model: {config.model_name}")
        _global_model_manager = ModelManager(config)
        _global_model_manager.load_model()
        _global_model_config = config
        return _global_model_manager

    if _global_model_config.model_name == config.model_name:
        print(f"Reusing existing model: {config.model_name}")
        return _global_model_manager

    print(f"Switching model: {_global_model_config.model_name} -> {config.model_name}")
    cleanup_model()
    _global_model_manager = ModelManager(config)
    _global_model_manager.load_model()
    _global_model_config = config

    return _global_model_manager


def cleanup_model():
    """Release model from memory"""
    global _global_model_manager, _global_model_config

    if _global_model_manager is None:
        print("No model to cleanup")
        return

    model_name = _global_model_config.model_name if _global_model_config else "Unknown"
    print(f"Cleaning up model: {model_name}")

    del _global_model_manager
    _global_model_manager = None
    _global_model_config = None

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("GPU memory freed")


def get_current_model_info() -> Dict[str, any]:
    """Get current model information"""
    if _global_model_manager is None:
        return {"loaded": False, "model_name": None, "gpu_memory_allocated": 0}

    return {
        "loaded": True,
        "model_name": _global_model_config.model_name,
        "gpu_memory_allocated": torch.cuda.memory_allocated(0) / 1e9 if torch.cuda.is_available() else 0
    }


print("Global model management loaded")
print("  - get_or_create_model_manager(config): Get or create model")
print("  - cleanup_model(): Release model memory")
print("  - get_current_model_info(): Get model status")

Global model management loaded
  - get_or_create_model_manager(config): Get or create model
  - cleanup_model(): Release model memory
  - get_current_model_info(): Get model status


## 7. Quality Gate System

In [9]:
class QualityGate:
    """Quality validation system for generated summaries"""

    def __init__(self, config: PipelineConfig):
        self.config = config
        self.validation_stats = {
            "total_validations": 0,
            "passed_first_time": 0,
            "regenerated": 0,
            "failed_final": 0
        }

    def validate_summary(self, summary: str, summary_type: str = "video") -> Tuple[bool, List[str]]:
        """
        Validate summary quality.

        Returns:
            Tuple of (is_valid, failure_reasons)
        """
        if not self.config.enable_quality_gate:
            return True, []

        self.validation_stats["total_validations"] += 1
        failure_reasons = []

        # Check 1: Minimum length
        if len(summary) < self.config.min_summary_length:
            failure_reasons.append(f"Too short ({len(summary)} < {self.config.min_summary_length} chars)")

        # Check 2: Keyword diversity
        words = summary.lower().split()
        unique_words = set(words)
        if len(unique_words) < self.config.min_keyword_diversity:
            failure_reasons.append(f"Low diversity ({len(unique_words)} < {self.config.min_keyword_diversity} unique words)")

        # Check 3: Content relevance
        if summary_type == "video":
            relevance_keywords = ["video", "content", "shows", "features", "presents", "discusses"]
        else:
            relevance_keywords = ["comment", "viewer", "audience", "reaction", "opinion", "sentiment"]

        if not any(keyword in summary.lower() for keyword in relevance_keywords):
            failure_reasons.append(f"Missing relevance keywords for {summary_type}")

        # Check 4: Required structure
        if "KEY POINT" not in summary:
            failure_reasons.append("Missing 'KEY POINT' section")

        # Check 5: Exact error message detection
        actual_failure_messages = [
            "Video summary generation failed.",
            "Reaction summary generation failed.",
        ]
        if summary.strip() in actual_failure_messages:
            failure_reasons.append("Generation returned error message")

        is_valid = len(failure_reasons) == 0
        if is_valid:
            self.validation_stats["passed_first_time"] += 1

        return is_valid, failure_reasons

    def get_quality_report(self) -> str:
        """Generate quality validation report"""
        stats = self.validation_stats
        total = stats["total_validations"]

        if total == 0:
            return "\n[Quality Gate: No validations performed]\n"

        pass_rate = (stats["passed_first_time"] / total * 100) if total > 0 else 0
        regen_rate = (stats["regenerated"] / total * 100) if total > 0 else 0

        return f"""
╔══════════════════════════════════════════════════════════╗
║              QUALITY GATE REPORT                         ║
╠══════════════════════════════════════════════════════════╣
║ Total Validations:     {total:>6}                         ║
║ Passed First Time:     {stats['passed_first_time']:>6} ({pass_rate:>5.1f}%)              ║
║ Regenerated:           {stats['regenerated']:>6} ({regen_rate:>5.1f}%)              ║
║ Failed Final:          {stats['failed_final']:>6}                         ║
╚══════════════════════════════════════════════════════════╝
"""

print("QualityGate loaded")

QualityGate loaded


## 8. Data Loading Utilities

In [10]:
def load_test_data(data_path: str, num_videos: int = 3) -> List[Dict]:
    """
    Load test data from JSON file.

    Args:
        data_path: Path to JSON data file
        num_videos: Number of videos to load

    Returns:
        List of video data dictionaries
    """
    print(f"Loading data from: {data_path}")

    with open(data_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Handle different data structures
    if isinstance(data, list):
        videos = data
    elif isinstance(data, dict) and 'videos' in data:
        videos = data['videos']
    else:
        raise ValueError(f"Unexpected data structure: {type(data)}")

    print(f"  Found {len(videos)} videos in dataset")
    test_videos = videos[:num_videos]
    print(f"  Using {len(test_videos)} videos for testing")

    return test_videos


print("Data loading utilities loaded")

Data loading utilities loaded


## 9. Sentiment Model Helper

Helper function to prepare sentiment model from zip file.

In [11]:
import zipfile

def ensure_sentiment_model_from_zip(zip_path: str, root_dir: str) -> str:
    """
    Extract sentiment model from zip if needed.

    Args:
        zip_path: Path to zip file containing bert_sentiment folder
        root_dir: Root directory to extract to

    Returns:
        Path to extracted model directory
    """
    target_dir = os.path.join(root_dir, "bert_sentiment")

    print("=" * 60)
    print("Preparing sentiment model from zip")
    print("=" * 60)

    if os.path.isdir(target_dir):
        print(f"Model directory already exists: {target_dir}")
        print(f"  Files: {os.listdir(target_dir)}")
        return target_dir

    if not os.path.exists(zip_path):
        print(f"Zip not found: {zip_path}")
        return target_dir

    os.makedirs(root_dir, exist_ok=True)

    print(f"Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, "r") as zf:
        print(f"  Size: {os.path.getsize(zip_path) / (1024**2):.1f} MB")
        print(f"  Files in zip: {len(zf.infolist())}")
        zf.extractall(root_dir)

    if os.path.isdir(target_dir):
        files = os.listdir(target_dir)
        print(f"Extracted to: {target_dir}")
        print(f"  Files ({len(files)}): {files}")
        missing = [f for f in ["config.json", "tokenizer_config.json"] if f not in files]
        if missing:
            print(f"  Warning: Missing required files: {missing}")
    else:
        print(f"Warning: {target_dir} not found after unzip")

    return target_dir


print("Sentiment model helper loaded")

Sentiment model helper loaded


## 10. Part 1 Verification

In [12]:
print("=" * 60)
print("Part 1 Setup Complete")
print("=" * 60)
print("")
print("Available components:")
print("  - PipelineConfig: Configuration dataclass")
print("  - config: Global configuration instance")
print("  - PromptTemplates: Prompt generation utilities")
print("  - TextPreprocessor: Text cleaning and language detection")
print("  - ModelManager: LLM model wrapper")
print("  - QualityGate: Output quality validation")
print("  - load_test_data(): Load video data from JSON")
print("  - ensure_sentiment_model_from_zip(): Prepare sentiment model")
print("  - get_or_create_model_manager(): Global model management")
print("  - cleanup_model(): Release model memory")
print("")
print("Next steps:")
print("  1. Run Part 2 to load Category and Sentiment models")
print("  2. Run Part 3 to execute the full pipeline")
print("=" * 60)

Part 1 Setup Complete

Available components:
  - PipelineConfig: Configuration dataclass
  - config: Global configuration instance
  - PromptTemplates: Prompt generation utilities
  - TextPreprocessor: Text cleaning and language detection
  - ModelManager: LLM model wrapper
  - QualityGate: Output quality validation
  - load_test_data(): Load video data from JSON
  - ensure_sentiment_model_from_zip(): Prepare sentiment model
  - get_or_create_model_manager(): Global model management
  - cleanup_model(): Release model memory

Next steps:
  1. Run Part 2 to load Category and Sentiment models
  2. Run Part 3 to execute the full pipeline



---

# From: part2_models.ipynb

---


# YouTube Report Generator - Part 2: Models (Category & Sentiment)

This notebook contains Category and Sentiment model definitions and initialization.

**Pipeline Structure:**
- **Part 1**: Setup, configuration, utilities, data loading
- **Part 2** (this file): Category and Sentiment model initialization
- **Part 3**: Full pipeline execution and report generation

**Prerequisites:** Run all cells in Part 1 before running this notebook.

**Note:** This notebook uses `config` (PipelineConfig instance) from Part 1.

## 1. Category Model

In [13]:
# NOTE: These imports are also in Part 1, but included here for standalone execution
import os
import pickle
import joblib
import torch
import numpy as np
from typing import List, Dict, Optional
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [14]:
class CategoryModelManager:
    """Category prediction model manager (BERT + TF-IDF)"""

    CATEGORY_MAPPING = {
        10: 'Music', 17: 'Sports', 19: 'Travel', 20: 'Gaming',
        22: 'People & Blogs', 23: 'Comedy', 24: 'Entertainment',
        25: 'News & Politics', 26: 'HowTo & Style',
        27: 'Education', 28: 'Science & Tech'
    }

    def __init__(self, model_dir: str = "./models"):
        self.model_dir = model_dir
        self.bert_model = None
        self.bert_tokenizer = None
        self.tfidf_model = None
        self.tfidf_vectorizer = None
        self.label_encoder = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"CategoryModelManager initialized (Device: {self.device})")

    def load_models(self) -> bool:
        """Load saved models"""
        if not os.path.exists(self.model_dir):
            print(f"Model directory not found: {self.model_dir}")
            return False

        try:
            # Load BERT model
            bert_path = os.path.join(self.model_dir, "bert_model")
            if os.path.exists(bert_path):
                self.bert_tokenizer = AutoTokenizer.from_pretrained(bert_path)
                self.bert_model = AutoModelForSequenceClassification.from_pretrained(
                    bert_path, device_map="auto"
                ).to(self.device)
                self.bert_model.eval()
                print(f"BERT model loaded: {bert_path}")
            else:
                print(f"BERT model not found: {bert_path}")

            # Load TF-IDF model
            tfidf_vec_path = os.path.join(self.model_dir, "tfidf_vectorizer.pkl")
            tfidf_model_path = os.path.join(self.model_dir, "tfidf_model.pkl")
            le_path = os.path.join(self.model_dir, "label_encoder.pkl")

            if all(os.path.exists(p) for p in [tfidf_vec_path, tfidf_model_path, le_path]):
                self.tfidf_vectorizer = joblib.load(tfidf_vec_path)
                self.tfidf_model = joblib.load(tfidf_model_path)
                self.label_encoder = joblib.load(le_path)
                print("TF-IDF model loaded")
            else:
                print("TF-IDF model files not found")

            return True

        except Exception as e:
            print(f"Model loading failed: {str(e)}")
            return False

    def save_models(self, bert_model, bert_tokenizer, tfidf_model,
                    tfidf_vectorizer, label_encoder):
        """Save trained models"""
        os.makedirs(self.model_dir, exist_ok=True)

        # Save BERT model
        bert_path = os.path.join(self.model_dir, "bert_model")
        bert_model.save_pretrained(bert_path)
        bert_tokenizer.save_pretrained(bert_path)
        print(f"BERT model saved: {bert_path}")

        # Save TF-IDF model
        joblib.dump(tfidf_vectorizer, os.path.join(self.model_dir, "tfidf_vectorizer.pkl"))
        joblib.dump(tfidf_model, os.path.join(self.model_dir, "tfidf_model.pkl"))
        joblib.dump(label_encoder, os.path.join(self.model_dir, "label_encoder.pkl"))
        print(f"TF-IDF model saved: {self.model_dir}")

        # Update internal references
        self.bert_model = bert_model
        self.bert_tokenizer = bert_tokenizer
        self.tfidf_model = tfidf_model
        self.tfidf_vectorizer = tfidf_vectorizer
        self.label_encoder = label_encoder

    def predict(self, text: str, use_bert: bool = True) -> str:
        """Predict category for given text"""
        if not text or len(text.strip()) < 5:
            return "Unknown"

        # Try BERT first
        if use_bert and self.bert_model is not None:
            try:
                inputs = self.bert_tokenizer(
                    text,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                    max_length=256
                ).to(self.device)

                with torch.no_grad():
                    outputs = self.bert_model(**inputs)
                    pred_idx = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

                return self.label_encoder.inverse_transform([pred_idx])[0]

            except Exception as e:
                print(f"BERT prediction failed, trying TF-IDF: {str(e)}")

        # Fallback to TF-IDF
        if self.tfidf_model is not None:
            try:
                text_vec = self.tfidf_vectorizer.transform([text])
                pred_idx = self.tfidf_model.predict(text_vec)[0]
                return self.label_encoder.inverse_transform([pred_idx])[0]
            except Exception as e:
                print(f"TF-IDF prediction failed: {str(e)}")

        return "Unknown"

    def get_status(self) -> Dict:
        """Get current model status"""
        return {
            "bert_loaded": self.bert_model is not None,
            "tfidf_loaded": self.tfidf_model is not None,
            "label_encoder_loaded": self.label_encoder is not None,
            "device": self.device
        }


print("CategoryModelManager class defined")

CategoryModelManager class defined


### 1.1 Category Model Initialization

**Mode options:**
- `load`: Load pre-trained model (default, fast)
- `train_script`: Run external training script (final_code.py)

In [15]:
print("=" * 60)
print("Category Model Initialization")
print("=" * 60)

# Create CategoryModelManager instance
category_model_manager = CategoryModelManager(model_dir="/content/drive/MyDrive/25_sch/ML/models")

# Mode selection: "load" or "train_script"
CATEGORY_MODE = "load"

print(f"\nSelected mode: {CATEGORY_MODE}")
print("=" * 60)

if CATEGORY_MODE == "load":
    print("\nLoading saved models...")
    success = category_model_manager.load_models()

    if success:
        print("\nModel load complete!")
        status = category_model_manager.get_status()
        print(f"  BERT: {'OK' if status['bert_loaded'] else 'Not loaded'}")
        print(f"  TF-IDF: {'OK' if status['tfidf_loaded'] else 'Not loaded'}")
        print(f"  Device: {status['device']}")
    else:
        print("\nModel load failed!")
        print("  Solution: Set CATEGORY_MODE='train_script' to train the model")

elif CATEGORY_MODE == "train_script":
    print("\nRunning final_code.py... (~30 min)")
    %run final_code.py

    print("\nSaving trained models...")
    try:
        MODEL_DIR = "./models"
        os.makedirs(MODEL_DIR, exist_ok=True)

        bert_save_path = os.path.join(MODEL_DIR, "bert_model")
        model_bert.save_pretrained(bert_save_path)
        tokenizer.save_pretrained(bert_save_path)
        print(f"BERT model saved: {bert_save_path}")

        joblib.dump(tfidf, os.path.join(MODEL_DIR, "tfidf_vectorizer.pkl"))
        joblib.dump(model_lr, os.path.join(MODEL_DIR, "tfidf_model.pkl"))
        joblib.dump(le, os.path.join(MODEL_DIR, "label_encoder.pkl"))
        print(f"TF-IDF model saved: {MODEL_DIR}")

        print("\nLoading saved models...")
        category_model_manager.load_models()
        print("Save and load complete!")

    except NameError as e:
        print(f"Model variables not found: {e}")
        print("  Check if final_code.py executed correctly.")
    except Exception as e:
        print(f"Save failed: {e}")

else:
    print(f"Unknown mode: {CATEGORY_MODE}")
    print("  Use 'load' or 'train_script'")

print("\n" + "=" * 60)

Category Model Initialization
CategoryModelManager initialized (Device: cuda)

Selected mode: load

Loading saved models...


The tokenizer you are loading from '/content/drive/MyDrive/25_sch/ML/models/bert_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


BERT model loaded: /content/drive/MyDrive/25_sch/ML/models/bert_model
TF-IDF model loaded

Model load complete!
  BERT: OK
  TF-IDF: OK
  Device: cuda



## 2. Sentiment Model

In [16]:
class SentimentModelManager:
    """Sentiment prediction model manager (neg/neu/pos)"""

    def __init__(self, model_dir: str):
        self.model_dir = model_dir
        self.tokenizer = None
        self.model = None
        self.bert_model = None  # Alias for compatibility
        self.classic_clf = None  # For compatibility check
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.labels = ["neg", "neu", "pos"]
        self.label2id = {"neg": 0, "neu": 1, "pos": 2}
        self.id2label = {0: "neg", 1: "neu", 2: "pos"}
        print(f"SentimentModelManager initialized (Device: {self.device})")

    def load_models(self) -> bool:
        """Load BERT sentiment model"""
        if not os.path.isdir(self.model_dir):
            print(f"Sentiment model directory not found: {self.model_dir}")
            return False

        try:
            print(f"  Loading from: {self.model_dir}")
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_dir)
            self.model = AutoModelForSequenceClassification.from_pretrained(
                self.model_dir
            ).to(self.device)
            self.model.eval()
            self.bert_model = self.model  # Alias for compatibility
            print("  Sentiment model loaded successfully!")
            return True

        except Exception as e:
            print(f"  Failed to load sentiment model: {e}")
            return False

    def predict(self, texts, use_bert: bool = True, use_classic: bool = True,
                ensemble: str = "bert_priority", batch_size: int = 32) -> Dict:
        """
        Predict sentiment for texts.

        Args:
            texts: Single text or list of texts
            use_bert: Use BERT model (ignored, always uses BERT)
            use_classic: Use classic model (ignored, for compatibility)
            ensemble: Ensemble method (ignored, for compatibility)
            batch_size: Batch size for inference

        Returns:
            Dict with 'predictions' (list of labels) and 'probabilities' (numpy array)
        """
        if isinstance(texts, str):
            texts = [texts]

        if self.model is None:
            raise RuntimeError("Sentiment model not loaded. Call load_models() first.")

        try:
            all_probs = []
            all_preds = []

            for i in range(0, len(texts), batch_size):
                batch_texts = texts[i:i+batch_size]
                inputs = self.tokenizer(
                    batch_texts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=128,
                ).to(self.device)

                with torch.no_grad():
                    outputs = self.model(**inputs)
                    probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
                    preds = np.argmax(probs, axis=1)

                all_probs.extend(probs)
                all_preds.extend(preds)

            proba = np.array(all_probs)
            pred_labels = [self.labels[p] for p in all_preds]

            return {
                "predictions": pred_labels,
                "probabilities": proba
            }

        except Exception as e:
            print(f"  Prediction error: {e}")
            return {
                "predictions": ["neu"] * len(texts),
                "probabilities": np.ones((len(texts), 3)) / 3
            }

    def get_status(self) -> Dict:
        """Get model status"""
        return {
            "loaded": self.model is not None,
            "device": self.device,
            "model_dir": self.model_dir,
        }


print("SentimentModelManager class defined")

SentimentModelManager class defined


### 2.1 Sentiment Model Initialization

**Mode options:**
- `load`: Load pre-trained model from zip (default)
- `train_script`: Run Bert_model.py training script

In [17]:
import zipfile

def ensure_sentiment_model_from_zip(zip_path: str, root_dir: str) -> str:
    """
    Extract sentiment model from zip if needed.

    Args:
        zip_path: Path to zip file containing bert_sentiment folder
        root_dir: Root directory to extract to

    Returns:
        Path to extracted model directory
    """
    target_dir = os.path.join(root_dir, "bert_sentiment")

    print("=" * 60)
    print("Preparing sentiment model from zip")
    print("=" * 60)

    if os.path.isdir(target_dir):
        print(f"Model directory already exists: {target_dir}")
        print(f"  Files: {os.listdir(target_dir)}")
        return target_dir

    if not os.path.exists(zip_path):
        print(f"Zip not found: {zip_path}")
        return target_dir

    os.makedirs(root_dir, exist_ok=True)

    print(f"Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, "r") as zf:
        print(f"  Size: {os.path.getsize(zip_path) / (1024**2):.1f} MB")
        print(f"  Files in zip: {len(zf.infolist())}")
        zf.extractall(root_dir)

    if os.path.isdir(target_dir):
        files = os.listdir(target_dir)
        print(f"Extracted to: {target_dir}")
        print(f"  Files ({len(files)}): {files}")
        missing = [f for f in ["config.json", "tokenizer_config.json"] if f not in files]
        if missing:
            print(f"  Warning: Missing required files: {missing}")
    else:
        print(f"Warning: {target_dir} not found after unzip")

    return target_dir

In [18]:
print("=" * 60)
print("Sentiment Model Initialization")
print("=" * 60)

# Mode selection: "load" or "train_script"
SENTIMENT_MODE = "load"

print(f"\nSelected mode: {SENTIMENT_MODE}")
print("-" * 60)

# Step 1: Prepare model from zip (common for load mode)
sentiment_model_dir = ensure_sentiment_model_from_zip(
    config.sentiment_zip_path,
    config.sentiment_root_dir,
)

# Step 2: Create SentimentModelManager
sentiment_model_manager = SentimentModelManager(
    model_dir=config.sentiment_model_path
)

# Step 3: Mode-specific processing
if SENTIMENT_MODE == "load":
    print("\nLoading pre-trained sentiment model...")
    print("-" * 60)

    success = sentiment_model_manager.load_models()
    status = sentiment_model_manager.get_status()

    if success and status["loaded"]:
        print("\n" + "=" * 60)
        print("Sentiment Model Loaded Successfully!")
        print("=" * 60)
        print(f"  Device: {status['device']}")
        print(f"  Model Dir: {status['model_dir']}")

        # Quick test
        print("\nQuick Test:")
        test_texts = ["정말 최고예요! 👍", "별로네.. 시간낭비", "그냥 보통이에요"]
        results = sentiment_model_manager.predict(test_texts)

        for text, pred, probs in zip(test_texts, results['predictions'], results['probabilities']):
            emoji = {"pos": "😊", "neu": "😐", "neg": "😠"}[pred]
            p_neg, p_neu, p_pos = probs
            print(f"  {emoji} '{text}' -> {pred} (pos={p_pos:.2f}, neu={p_neu:.2f}, neg={p_neg:.2f})")

        print("=" * 60)
    else:
        print("\n" + "=" * 60)
        print("Failed to load Sentiment model")
        print("=" * 60)
        print("Troubleshooting:")
        print("  1. Check zip path: config.sentiment_zip_path")
        print("  2. Check extraction: config.sentiment_model_path")
        print("  3. Required files: config.json, model.safetensors, tokenizer")
        print("  4. Or set SENTIMENT_MODE='train_script' to train the model")
        print("=" * 60)

elif SENTIMENT_MODE == "train_script":
    print("\ntrain_script mode: running Bert_model.py")
    print("-" * 60)

    if not os.path.exists("Bert_model.py"):
        print("\nBert_model.py not found")
        print("  Download from: https://github.com/myWworld/20252R0136COSE36203")
    else:
        print("Bert_model.py found")
        print("\nRunning Bert_model.py...")
        print("=" * 60)

        %run Bert_model.py

        print("\n" + "=" * 60)
        print("Bert_model.py execution complete!")
        print("=" * 60)

        # Move trained model if needed
        if os.path.isdir("./bert_sentiment"):
            target_dir = config.sentiment_model_path
            if "./bert_sentiment" != target_dir:
                import shutil
                os.makedirs(os.path.dirname(target_dir), exist_ok=True)
                if not os.path.isdir(target_dir):
                    shutil.move("./bert_sentiment", target_dir)
                    print(f"Moved: ./bert_sentiment -> {target_dir}")
                else:
                    print(f"{target_dir} already exists, skipping move")

        print("\nLoading trained sentiment model...")
        print("-" * 60)

        success = sentiment_model_manager.load_models()

        if success:
            print("Sentiment model ready after train_script!")

            print("\nQuick Test:")
            test_texts = ["정말 최고예요! 👍", "별로네.. 시간낭비", "그냥 보통이에요"]
            results = sentiment_model_manager.predict(test_texts)

            for text, pred in zip(test_texts, results['predictions']):
                emoji = {"pos": "😊", "neu": "😐", "neg": "😠"}[pred]
                print(f"  {emoji} '{text}' -> {pred}")
        else:
            print("Sentiment model load failed after training.")
            print("  Check Bert_model.py execution logs")

else:
    print(f"\nUnknown mode: {SENTIMENT_MODE}")
    print("  Use 'load' or 'train_script'")

print("\n" + "=" * 60)
print("Sentiment Model Initialization Complete!")
print("=" * 60)

Sentiment Model Initialization

Selected mode: load
------------------------------------------------------------
Preparing sentiment model from zip
Extracting /content/drive/MyDrive/25_sch/ML/sentiment/bert_sentiment.zip...
  Size: 805.6 MB
  Files in zip: 7
Extracted to: ./models/sentiment/bert_sentiment
  Files (7): ['training_args.bin', 'tokenizer.json', 'special_tokens_map.json', 'model.safetensors', 'tokenizer_config.json', 'config.json', 'sentencepiece.bpe.model']
SentimentModelManager initialized (Device: cuda)

Loading pre-trained sentiment model...
------------------------------------------------------------
  Loading from: ./models/sentiment/bert_sentiment


The tokenizer you are loading from './models/sentiment/bert_sentiment' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


  Sentiment model loaded successfully!

Sentiment Model Loaded Successfully!
  Device: cuda
  Model Dir: ./models/sentiment/bert_sentiment

Quick Test:
  😊 '정말 최고예요! 👍' -> pos (pos=1.00, neu=0.00, neg=0.00)
  😠 '별로네.. 시간낭비' -> neg (pos=0.01, neu=0.01, neg=0.98)
  😐 '그냥 보통이에요' -> neu (pos=0.01, neu=0.98, neg=0.01)

Sentiment Model Initialization Complete!


## 3. Optional: Clear Sentiment Model Directory

Run this cell only if you need to re-extract the sentiment model.

In [ ]:
# Uncomment and run only if needed
# import shutil
# folder_to_delete = "/content/models/sentiment"
# if os.path.exists(folder_to_delete):
#     shutil.rmtree(folder_to_delete)
#     print(f"Deleted: {folder_to_delete}")
# else:
#     print(f"Folder not found: {folder_to_delete}")

## 4. Part 2 Verification

In [19]:
print("=" * 60)
print("Part 2 Model Setup Complete")
print("=" * 60)
print("")
print("Loaded models:")

# Category model status
if 'category_model_manager' in globals():
    cat_status = category_model_manager.get_status()
    print(f"  Category Model:")
    print(f"    BERT: {'OK' if cat_status['bert_loaded'] else 'Not loaded'}")
    print(f"    TF-IDF: {'OK' if cat_status['tfidf_loaded'] else 'Not loaded'}")
else:
    print("  Category Model: Not initialized")

# Sentiment model status
if 'sentiment_model_manager' in globals():
    sent_status = sentiment_model_manager.get_status()
    print(f"  Sentiment Model:")
    print(f"    Loaded: {'OK' if sent_status['loaded'] else 'Not loaded'}")
    print(f"    Device: {sent_status['device']}")
else:
    print("  Sentiment Model: Not initialized")

print("")
print("Global variables available:")
print("  - category_model_manager: CategoryModelManager instance")
print("  - sentiment_model_manager: SentimentModelManager instance")
print("")
print("Next steps:")
print("  Run Part 3 to execute the full pipeline")
print("=" * 60)

Part 2 Model Setup Complete

Loaded models:
  Category Model:
    BERT: OK
    TF-IDF: OK
  Sentiment Model:
    Loaded: OK
    Device: cuda

Global variables available:
  - category_model_manager: CategoryModelManager instance
  - sentiment_model_manager: SentimentModelManager instance

Next steps:
  Run Part 3 to execute the full pipeline



---

# From: part3_pipeline_report.ipynb

---


# YouTube Report Generator - Part 3: Full Pipeline & Report

This notebook contains the ReportGenerator class and pipeline execution.

**Pipeline Structure:**
- **Part 1**: Setup, configuration, utilities, data loading
- **Part 2**: Category and Sentiment model initialization
- **Part 3** (this file): Full pipeline execution and report generation

**Prerequisites:** Run all cells in Part 1 and Part 2 before running this notebook.

**Required global variables from previous parts:**
- `config` (PipelineConfig)
- `category_model_manager` (CategoryModelManager)
- `sentiment_model_manager` (SentimentModelManager)
- Utility classes: `TextPreprocessor`, `PromptTemplates`, `ModelManager`, `QualityGate`
- Functions: `get_or_create_model_manager`, `load_test_data`

## 1. Report Generator

In [20]:
# NOTE: These imports are for standalone execution reference
from datetime import datetime
from typing import List, Dict, Optional, Tuple
from collections import Counter

In [21]:
class ReportGenerator:
    """Report generator with quality gate integration"""

    CATEGORY_MAPPING = {
        '10': 'Music', '17': 'Sports', '19': 'Travel', '20': 'Gaming',
        '22': 'People & Blogs', '23': 'Comedy', '24': 'Entertainment',
        '25': 'News & Politics', '26': 'HowTo & Style',
        '27': 'Education', '28': 'Science & Tech',
        10: 'Music', 17: 'Sports', 19: 'Travel', 20: 'Gaming',
        22: 'People & Blogs', 23: 'Comedy', 24: 'Entertainment',
        25: 'News & Politics', 26: 'HowTo & Style',
        27: 'Education', 28: 'Science & Tech'
    }

    def __init__(self, config: 'PipelineConfig', model_manager: 'ModelManager'):
        self.config = config
        self.model = model_manager
        self.quality_gate = QualityGate(config)

    def predict_category(self, video_data: Dict) -> Dict[str, any]:
        """
        Predict category using ML model and LLM validation.

        Returns:
            Dict with original_id, original_name, model_prediction, llm_validated, used_llm
        """
        video_info = video_data.get('video_info', {})
        original_id = video_info.get('category_id', 'Unknown')

        result = {
            'original_id': original_id,
            'original_name': self.CATEGORY_MAPPING.get(original_id, 'Unknown'),
            'model_prediction': None,
            'llm_validated': None,
            'used_llm': False
        }

        # Step 1: ML Model prediction
        predicted_category_id = original_id
        if self.config.use_category_model and 'category_model_manager' in globals():
            try:
                title = video_info.get('title', '')
                description = video_info.get('description', '')
                text = f"{title} {description}"

                predicted_name = category_model_manager.predict(text, use_bert=True)
                result['model_prediction'] = predicted_name
                print(f"  ML Model Prediction: {predicted_name}")

                # Reverse mapping: name -> id
                name_to_id = {v: k for k, v in self.CATEGORY_MAPPING.items() if isinstance(k, int)}
                predicted_category_id = name_to_id.get(predicted_name, original_id)

            except Exception as e:
                print(f"  ML Model prediction failed: {str(e)}")
                result['model_prediction'] = "Model Error"
        else:
            print("  Category Model disabled or not loaded")
            result['model_prediction'] = "Not Used"

        # Step 2: LLM validation
        if self.config.enable_llm_category_correction:
            try:
                if isinstance(predicted_category_id, str):
                    predicted_category_id = int(predicted_category_id)

                validated_id = self.validate_category(video_data, predicted_category_id)
                result['llm_validated'] = self.CATEGORY_MAPPING.get(validated_id, f"ID:{validated_id}")
                result['used_llm'] = True
                print(f"  LLM Validated: {result['llm_validated']}")

            except Exception as e:
                print(f"  LLM validation failed: {str(e)}")
                result['llm_validated'] = result['model_prediction'] or result['original_name']
                result['used_llm'] = False
        else:
            result['llm_validated'] = result['model_prediction'] or result['original_name']
            result['used_llm'] = False
            print("  LLM Category Correction disabled")

        return result

    def validate_category(self, video_data: Dict, predicted_category: int) -> int:
        """Validate category using LLM"""
        if not self.config.enable_llm_category_correction:
            return predicted_category

        print("  Validating category with LLM...")

        video_info = video_data.get('video_info', {})
        title = video_info.get('title', '')
        description = video_info.get('description', '')

        prompt = PromptTemplates.get_category_correction_prompt(
            title, description, predicted_category
        )

        try:
            response = self.model.generate(prompt, temperature=0.3)
            response = response.strip().upper()

            if "CORRECT:" in response:
                corrected = int(response.split(":")[1].strip())
                if corrected != predicted_category:
                    print(f"    Category corrected: {predicted_category} -> {corrected}")
                    return corrected
                else:
                    print(f"    Category {predicted_category} validated")
                    return predicted_category

        except Exception as e:
            print(f"    Validation failed, using original: {str(e)}")

        return predicted_category

    def generate_video_summary(self, video_data: Dict) -> Tuple[str, str]:
        """Generate video summary with quality validation"""
        print("  Generating video summary...")

        video_info = video_data.get('video_info', {})
        title = video_info.get('title', 'N/A')
        description = video_info.get('description', 'N/A')
        category_id = video_info.get('category_id', 'Unknown')
        duration = video_info.get('duration', 'Unknown')

        # Preprocess description
        if self.config.remove_urls:
            description = TextPreprocessor.remove_urls(description)
        if len(description) > self.config.max_description_length:
            description = description[:self.config.max_description_length] + "..."

        # Detect language
        detected_lang = "unknown"
        if self.config.detect_language:
            detected_lang = TextPreprocessor.detect_language(f"{title} {description}")
            print(f"    [Input language: {detected_lang}]")

        prompt = PromptTemplates.get_video_summary_prompt(
            title, description, str(category_id), str(duration),
            self.config.output_language
        )

        # Quality gate with regeneration
        summary = None
        attempts = 0
        max_attempts = self.config.max_regeneration_attempts + 1

        while attempts < max_attempts:
            try:
                temp = self.config.quality_gate_temperature if attempts > 0 else self.config.temperature
                summary = self.model.generate(prompt, temperature=temp)

                is_valid, failure_reasons = self.quality_gate.validate_summary(summary, "video")

                if is_valid:
                    if attempts > 0:
                        print(f"    Quality passed after {attempts} regeneration(s)")
                        self.quality_gate.validation_stats["regenerated"] += 1
                    break
                else:
                    attempts += 1
                    if attempts < max_attempts:
                        print(f"    Quality check failed (attempt {attempts}): {', '.join(failure_reasons)}")
                        print(f"    Regenerating...")
                    else:
                        print(f"    Quality check failed after {attempts} attempts")
                        self.quality_gate.validation_stats["failed_final"] += 1

            except Exception as e:
                print(f"    Error: {str(e)}")
                summary = "Video summary generation failed."
                break

        output_lang = TextPreprocessor.detect_language(summary)
        print(f"    [Output language: {output_lang}]")

        return summary, detected_lang

    def generate_reaction_summary(self, comments: List[str]) -> Tuple[str, Dict[str, int]]:
        """Generate reaction summary with quality validation"""
        print("  Generating reaction summary...")

        if not comments:
            return "No comments available for analysis.", {}

        # Preprocess and filter comments
        processed_comments = []
        for comment in comments[:self.config.max_comments_to_process]:
            cleaned = TextPreprocessor.clean_text(comment, remove_urls=self.config.remove_urls)
            if len(cleaned) >= self.config.min_comment_length:
                processed_comments.append(cleaned)

        if not processed_comments:
            return "No valid comments after filtering.", {}

        # Language distribution
        lang_dist = TextPreprocessor.get_language_distribution(processed_comments)
        print(f"    [Comment languages: {lang_dist}]")

        prompt = PromptTemplates.get_reaction_summary_prompt(
            processed_comments,
            self.config.output_language
        )

        # Quality gate with regeneration
        summary = None
        attempts = 0
        max_attempts = self.config.max_regeneration_attempts + 1

        while attempts < max_attempts:
            try:
                temp = self.config.quality_gate_temperature if attempts > 0 else self.config.temperature
                summary = self.model.generate(prompt, temperature=temp)

                is_valid, failure_reasons = self.quality_gate.validate_summary(summary, "reaction")

                if is_valid:
                    if attempts > 0:
                        print(f"    Quality passed after {attempts} regeneration(s)")
                        self.quality_gate.validation_stats["regenerated"] += 1
                    break
                else:
                    attempts += 1
                    if attempts < max_attempts:
                        print(f"    Quality check failed (attempt {attempts}): {', '.join(failure_reasons)}")
                        print(f"    Regenerating...")
                    else:
                        print(f"    Quality check failed after {attempts} attempts")
                        self.quality_gate.validation_stats["failed_final"] += 1

            except Exception as e:
                print(f"    Error: {str(e)}")
                summary = "Reaction summary generation failed."
                break

        output_lang = TextPreprocessor.detect_language(summary)
        print(f"    [Output language: {output_lang}]")

        return summary, lang_dist

    def calculate_metrics(self, video_data: Dict) -> Dict[str, float]:
        """Calculate engagement metrics"""
        print("  Calculating metrics...")

        views = video_data.get('view_count', 0)
        likes = video_data.get('like_count', 0)
        comments = video_data.get('comment_count', 0)

        return {
            'engagement_rate': ((likes + comments) / views * 100) if views > 0 else 0,
            'like_rate': (likes / views * 100) if views > 0 else 0,
            'comment_rate': (comments / views * 100) if views > 0 else 0
        }

    def format_report(self, video_data: Dict, video_summary: str,
                      reaction_summary: str, metrics: Dict,
                      video_lang: str, comment_langs: Dict,
                      sentiment_summary: Optional[Dict] = None) -> str:
        """Format the final markdown report"""
        print("  Formatting report...")

        video_info = video_data.get('video_info', {})
        category_info = self.predict_category(video_data)

        # Top comments
        top_comments = []
        for i, comment in enumerate(video_data.get('comments', [])[:5], 1):
            author = comment.get('author', 'Unknown')
            text = comment.get('text', '')
            likes = comment.get('like_count', 0)
            top_comments.append(f"{i}. **@{author}** ({likes} likes): {text[:100]}...")

        top_comments_section = "\n\n".join(top_comments) if top_comments else "No comments available."

        # Language formatting
        if comment_langs:
            sorted_langs = sorted(comment_langs.items(), key=lambda x: x[1], reverse=True)
            comment_lang_str = ", ".join([f"{lang} ({count})" for lang, count in sorted_langs])
        else:
            comment_lang_str = "No comments analyzed"

        video_lang_display = "Unknown" if video_lang.lower() == "unknown" else video_lang

        # Comment analysis stats
        total_comments = video_info.get('comment_count', 0)
        analyzed_comments = len(video_data.get('comments', []))
        analysis_rate = (analyzed_comments / total_comments * 100) if total_comments > 0 else 0

        # Category display
        if category_info['used_llm']:
            category_display = f"- **Category (ML Model)**: {category_info['model_prediction']} \n- **Category (LLM Validated)**: {category_info['llm_validated']}"
        elif category_info['model_prediction'] and category_info['model_prediction'] != "Not Used":
            category_display = f"- **Category (ML Model)**: {category_info['model_prediction']}"
        else:
            category_display = f"- **Category**: {category_info['original_id']} ({category_info['original_name']})"

        report = f"""# YouTube Video Report

**Generated**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Model**: {self.config.model_name}
**Pipeline Version**: 2.3

---

## Video Information

- **Title**: {video_info.get('title', 'N/A')}
- **Channel**: {video_info.get('channel_title', 'N/A')}
{category_display}
- **Published**: {video_info.get('published_at', 'N/A')}
- **Duration**: {video_info.get('duration', 'N/A')} seconds
- **Video ID**: `{video_info.get('video_id', 'N/A')}`
- **URL**: https://www.youtube.com/watch?v={video_info.get('video_id', 'N/A')}

### Detected Languages

- **Video content**: {video_lang_display}
- **Comments**: {comment_lang_str}
- **Report language**: {self.config.output_language}

---

## Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | {video_info.get('view_count', 0):,} |
| Likes | {video_info.get('like_count', 0):,} |
| Comments | {video_info.get('comment_count', 0):,} |
| Engagement Rate | {metrics.get('engagement_rate', 0):.2f}% |
| Like Rate | {metrics.get('like_rate', 0):.2f}% |
| Comment Rate | {metrics.get('comment_rate', 0):.2f}% |

---

## Video Summary

{video_summary}

---

## Audience Reaction Summary

**Comments Analyzed**: {analyzed_comments} / {total_comments} ({analysis_rate:.1f}%)

### ML Model Sentiment Analysis

"""

        # Add ML sentiment if available
        if sentiment_summary:
            report += f"""**Distribution:**
- Positive: {sentiment_summary['pos_count']} ({sentiment_summary['pos_ratio']:.1f}%)
- Neutral: {sentiment_summary['neu_count']} ({sentiment_summary['neu_ratio']:.1f}%)
- Negative: {sentiment_summary['neg_count']} ({sentiment_summary['neg_ratio']:.1f}%)

**Model Confidence (Average):**
- P(Positive): {sentiment_summary['avg_p_pos']:.1f}%
- P(Neutral): {sentiment_summary['avg_p_neu']:.1f}%
- P(Negative): {sentiment_summary['avg_p_neg']:.1f}%

"""
            # Interpretation
            if sentiment_summary['pos_ratio'] > 60:
                report += "**Interpretation**: Highly positive audience reception!\n\n"
            elif sentiment_summary['neg_ratio'] > 40:
                report += "**Interpretation**: Audience shows significant negative sentiment.\n\n"
            else:
                report += "**Interpretation**: Mixed audience sentiment with balanced reactions.\n\n"
        else:
            report += "*(ML Model not available or disabled)*\n\n"

        report += "---\n\n"

        # LLM analysis
        report += f"""### LLM Qualitative Analysis

{reaction_summary}

---

## Top Comments

{top_comments_section}

---

## Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- Token Efficiency: Dynamic token allocation based on input length
- Quality Gate: Automatic validation and regeneration for quality assurance
- Input content processed in original language(s) without translation layer
- Output language fixed to {self.config.output_language} for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.3*
"""

        return report

    def generate_full_report(self, video_data: Dict) -> str:
        """Generate complete report pipeline"""
        video_info = video_data.get('video_info', {})
        print(f"\nProcessing: {video_info.get('title', 'Unknown')[:50]}...")

        # 1. Video summary
        video_summary, video_lang = self.generate_video_summary(video_data)

        # 2. Sentiment analysis (ML Model)
        sentiment_summary = None
        if self.config.use_sentiment_model and 'sentiment_model_manager' in globals():
            if sentiment_model_manager.bert_model or sentiment_model_manager.classic_clf:
                print("\n  Analyzing sentiment with ML model...")

                comments = video_data.get('comments', [])
                if comments:
                    max_comments = min(len(comments), self.config.max_comments_to_process)
                    comment_texts = [c.get('text', '') for c in comments[:max_comments]]

                    try:
                        sentiment_results = sentiment_model_manager.predict(
                            comment_texts,
                            use_bert=True,
                            use_classic=True,
                            ensemble="bert_priority"
                        )

                        preds = sentiment_results["predictions"]
                        probs = sentiment_results["probabilities"]

                        pred_counts = Counter(preds)
                        total = len(preds)

                        sentiment_summary = {
                            "pos_count": pred_counts.get("pos", 0),
                            "neu_count": pred_counts.get("neu", 0),
                            "neg_count": pred_counts.get("neg", 0),
                            "pos_ratio": pred_counts.get("pos", 0) / total * 100 if total > 0 else 0,
                            "neu_ratio": pred_counts.get("neu", 0) / total * 100 if total > 0 else 0,
                            "neg_ratio": pred_counts.get("neg", 0) / total * 100 if total > 0 else 0,
                            "avg_p_pos": probs[:, 2].mean() * 100,
                            "avg_p_neu": probs[:, 1].mean() * 100,
                            "avg_p_neg": probs[:, 0].mean() * 100,
                            "total_analyzed": total
                        }

                        print(f"    ML Sentiment: Pos {sentiment_summary['pos_ratio']:.1f}% | "
                              f"Neu {sentiment_summary['neu_ratio']:.1f}% | "
                              f"Neg {sentiment_summary['neg_ratio']:.1f}%")

                    except Exception as e:
                        print(f"    Sentiment analysis failed: {e}")
                        sentiment_summary = None

        # 3. Reaction summary
        comments = [c.get('text', '') for c in video_data.get('comments', [])]
        reaction_summary, comment_langs = self.generate_reaction_summary(comments)

        # 4. Calculate metrics
        metrics = self.calculate_metrics(video_info)

        # 5. Format report
        report = self.format_report(
            video_data,
            video_summary,
            reaction_summary,
            metrics,
            video_lang,
            comment_langs,
            sentiment_summary
        )

        print("  Done!")
        return report


print("ReportGenerator loaded")

ReportGenerator loaded


## 2. Pipeline Execution Functions

In [22]:
def run_pipeline(config: 'PipelineConfig', test_videos: List[Dict],
                 model_manager: Optional['ModelManager'] = None,
                 force_reload: bool = False) -> List[str]:
    """
    Execute full pipeline on multiple videos.

    Args:
        config: Pipeline configuration
        test_videos: List of video data dictionaries
        model_manager: Existing model manager (None to auto-create)
        force_reload: Force reload model if True

    Returns:
        List of generated reports
    """
    print("=" * 80)
    print("STARTING FULL PIPELINE")
    print("=" * 80)

    # Get or create model manager
    if model_manager is None:
        model_manager = get_or_create_model_manager(config, force_reload)
    else:
        print("Using provided model manager")

    # Initialize report generator
    report_gen = ReportGenerator(config, model_manager)

    # Process each video
    reports = []
    for i, video in enumerate(test_videos, 1):
        print(f"\n{'=' * 80}")
        print(f"Video {i}/{len(test_videos)}")
        print(f"{'=' * 80}")

        report = report_gen.generate_full_report(video)
        reports.append(report)

    print(f"\n{'=' * 80}")
    print("PIPELINE COMPLETED")
    print("=" * 80)

    # Print efficiency reports
    print(model_manager.get_token_efficiency_report())
    print(report_gen.quality_gate.get_quality_report())

    return reports


print("Pipeline functions loaded")

Pipeline functions loaded


## 3. Execute Pipeline (JSON Data)

In [23]:
# Load test data and run pipeline
test_videos = load_test_data(config.data_path, config.num_videos_for_test)
reports = run_pipeline(config, test_videos)

Loading data from: /content/drive/MyDrive/25_sch/ML/youtube_test_10132154/full_dataset_20251013_215347.json
  Found 20 videos in dataset
  Using 3 videos for testing
STARTING FULL PIPELINE
Loading model: meta-llama/Llama-3.1-8B-Instruct
Loading meta-llama/Llama-3.1-8B-Instruct...


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


Model loaded on cuda
Memory allocated: 7.54 GB

Video 1/3

Processing: NMIXX(엔믹스) “Blue Valentine” M/V...
  Generating video summary...
    [Input language: English]
      [Input: 305 tokens, Max output: 256]
      [Output: 153 tokens]
    [Output language: English]

  Analyzing sentiment with ML model...
    ML Sentiment: Pos 100.0% | Neu 0.0% | Neg 0.0%
  Generating reaction summary...
    [Comment languages: {'Korean': 9, 'English': 8, 'Japanese': 3}]
      [Input: 1478 tokens, Max output: 384]
      [Output: 194 tokens]
    [Output language: English]
  Calculating metrics...
  Formatting report...
  ML Model Prediction: Gaming
  Validating category with LLM...
      [Input: 371 tokens, Max output: 256]
      [Output: 6 tokens]
    Category corrected: 20 -> 10
  LLM Validated: Music
  Done!

Video 2/3

Processing: How well do I know ROBLOX | ALBERTGUESSR...
  Generating video summary...
    [Input language: English]
      [Input: 260 tokens, Max output: 256]
      [Output: 179 token

In [24]:
# Display first report
from IPython.display import Markdown, display

print("=" * 80)
print("SAMPLE REPORT (First Video)")
print("=" * 80)
display(Markdown(reports[0]))

SAMPLE REPORT (First Video)


# YouTube Video Report

**Generated**: 2025-12-13 01:02:57
**Model**: meta-llama/Llama-3.1-8B-Instruct
**Pipeline Version**: 2.3

---

## Video Information

- **Title**: NMIXX(엔믹스) “Blue Valentine” M/V
- **Channel**: JYP Entertainment
- **Category (ML Model)**: Gaming 
- **Category (LLM Validated)**: Music
- **Published**: 2025-10-13T09:00:02Z
- **Duration**: PT3M15S seconds
- **Video ID**: `EmeW6li6bbo`
- **URL**: https://www.youtube.com/watch?v=EmeW6li6bbo

### Detected Languages

- **Video content**: English
- **Comments**: Korean (9), English (8), Japanese (3)
- **Report language**: English

---

## Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | 1,295,943 |
| Likes | 66,047 |
| Comments | 6,284 |
| Engagement Rate | 5.58% |
| Like Rate | 5.10% |
| Comment Rate | 0.48% |

---

## Video Summary

KEY POINT:
The music video "Blue Valentine" by NMIXX is a captivating visual representation of the K-pop group's energetic and emotive performance, featuring a blend of vibrant colors and synchronized choreography. The video showcases the group's unique style and musicality.

DETAILED SUMMARY:
"Blue Valentine" is a music video by the South Korean girl group NMIXX, released under JYP Entertainment. The video features the group's members performing an addictive and upbeat track, showcasing their synchronized dance moves and captivating stage presence. With a bright and vibrant color palette, the video adds to the overall energetic and youthful vibe of the song. Throughout the video, the members demonstrate their impressive dance skills and charismatic charm, highlighting the group's talent and appeal.

---

## Audience Reaction Summary

**Comments Analyzed**: 20 / 6284 (0.3%)

### ML Model Sentiment Analysis

**Distribution:**
- Positive: 20 (100.0%)
- Neutral: 0 (0.0%)
- Negative: 0 (0.0%)

**Model Confidence (Average):**
- P(Positive): 90.6%
- P(Neutral): 7.5%
- P(Negative): 1.9%

**Interpretation**: Highly positive audience reception!

---

### LLM Qualitative Analysis

KEY POINT:
The overall reaction to the NMIXX song and music video is overwhelmingly positive, with viewers praising the group's unique sound, emotional lyrics, and captivating visuals.

DETAILED SUMMARY:
The majority of viewers (around 85%) are thrilled with the song and music video, praising NMIXX for their growth and unique style. Many commenters (around 60%) highlight the emotional lyrics, vocal delivery, and nostalgic vibe of the song, which they believe will resonate with listeners. Some viewers (around 10%) appreciate the music video's captivating visuals and choreography, while others (around 5%) mention the song's ability to evoke various emotions, from sadness to happiness. Notable minority views include a few commenters who express concerns about the song's potential to appeal to a broader audience, but these views are relatively rare.

SENTIMENT BREAKDOWN:
Positive: 85%  Negative: 5%  Neutral: 10%

---

## Top Comments

1. **@@하-d7k** (58 likes): 이건 열심히 달려온 모든게 준비된 엔믹스들에게 온 너무 소중한 기회임 안 뜬다면… 케이팝 시장에 문제가 있는 건데 앞으로 홍보만 잘 돌리면 될 것 같은데 제발 힘을 내 제와피 !!...

2. **@@mikzzza-k7o** (141 likes): i have this unexplainable feeling that this song will be a hit, the biggest so far in the entire nmi...

3. **@@Belle_vie** (33 likes): 와 이번에는 수록곡뿐만 아니라 타이틀도 너무 잘 꼽았다 진짜… 트렌디하면서도 뭔가 애틋 아련하면서도 벅차고 느낌 너무 좋은데....

4. **@@mAsHuMaRoPanda** (132 likes): Wait! People! Seriously! Please pay attention to lyrics! Their vocal delivery! Like this is the most...

5. **@@rynnieeefairy** (320 likes): this song feels so upbeat and nostalgic!! and like many are saying, their sound/color is so unique, ...

---

## Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- Token Efficiency: Dynamic token allocation based on input length
- Quality Gate: Automatic validation and regeneration for quality assurance
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.3*


In [25]:
# Display all reports (optional)
for i, report in enumerate(reports, 1):
    print(f"\n{'=' * 80}")
    print(f"REPORT {i}/{len(reports)}")
    print("=" * 80)
    display(Markdown(report))


REPORT 1/3


# YouTube Video Report

**Generated**: 2025-12-13 01:02:57
**Model**: meta-llama/Llama-3.1-8B-Instruct
**Pipeline Version**: 2.3

---

## Video Information

- **Title**: NMIXX(엔믹스) “Blue Valentine” M/V
- **Channel**: JYP Entertainment
- **Category (ML Model)**: Gaming 
- **Category (LLM Validated)**: Music
- **Published**: 2025-10-13T09:00:02Z
- **Duration**: PT3M15S seconds
- **Video ID**: `EmeW6li6bbo`
- **URL**: https://www.youtube.com/watch?v=EmeW6li6bbo

### Detected Languages

- **Video content**: English
- **Comments**: Korean (9), English (8), Japanese (3)
- **Report language**: English

---

## Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | 1,295,943 |
| Likes | 66,047 |
| Comments | 6,284 |
| Engagement Rate | 5.58% |
| Like Rate | 5.10% |
| Comment Rate | 0.48% |

---

## Video Summary

KEY POINT:
The music video "Blue Valentine" by NMIXX is a captivating visual representation of the K-pop group's energetic and emotive performance, featuring a blend of vibrant colors and synchronized choreography. The video showcases the group's unique style and musicality.

DETAILED SUMMARY:
"Blue Valentine" is a music video by the South Korean girl group NMIXX, released under JYP Entertainment. The video features the group's members performing an addictive and upbeat track, showcasing their synchronized dance moves and captivating stage presence. With a bright and vibrant color palette, the video adds to the overall energetic and youthful vibe of the song. Throughout the video, the members demonstrate their impressive dance skills and charismatic charm, highlighting the group's talent and appeal.

---

## Audience Reaction Summary

**Comments Analyzed**: 20 / 6284 (0.3%)

### ML Model Sentiment Analysis

**Distribution:**
- Positive: 20 (100.0%)
- Neutral: 0 (0.0%)
- Negative: 0 (0.0%)

**Model Confidence (Average):**
- P(Positive): 90.6%
- P(Neutral): 7.5%
- P(Negative): 1.9%

**Interpretation**: Highly positive audience reception!

---

### LLM Qualitative Analysis

KEY POINT:
The overall reaction to the NMIXX song and music video is overwhelmingly positive, with viewers praising the group's unique sound, emotional lyrics, and captivating visuals.

DETAILED SUMMARY:
The majority of viewers (around 85%) are thrilled with the song and music video, praising NMIXX for their growth and unique style. Many commenters (around 60%) highlight the emotional lyrics, vocal delivery, and nostalgic vibe of the song, which they believe will resonate with listeners. Some viewers (around 10%) appreciate the music video's captivating visuals and choreography, while others (around 5%) mention the song's ability to evoke various emotions, from sadness to happiness. Notable minority views include a few commenters who express concerns about the song's potential to appeal to a broader audience, but these views are relatively rare.

SENTIMENT BREAKDOWN:
Positive: 85%  Negative: 5%  Neutral: 10%

---

## Top Comments

1. **@@하-d7k** (58 likes): 이건 열심히 달려온 모든게 준비된 엔믹스들에게 온 너무 소중한 기회임 안 뜬다면… 케이팝 시장에 문제가 있는 건데 앞으로 홍보만 잘 돌리면 될 것 같은데 제발 힘을 내 제와피 !!...

2. **@@mikzzza-k7o** (141 likes): i have this unexplainable feeling that this song will be a hit, the biggest so far in the entire nmi...

3. **@@Belle_vie** (33 likes): 와 이번에는 수록곡뿐만 아니라 타이틀도 너무 잘 꼽았다 진짜… 트렌디하면서도 뭔가 애틋 아련하면서도 벅차고 느낌 너무 좋은데....

4. **@@mAsHuMaRoPanda** (132 likes): Wait! People! Seriously! Please pay attention to lyrics! Their vocal delivery! Like this is the most...

5. **@@rynnieeefairy** (320 likes): this song feels so upbeat and nostalgic!! and like many are saying, their sound/color is so unique, ...

---

## Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- Token Efficiency: Dynamic token allocation based on input length
- Quality Gate: Automatic validation and regeneration for quality assurance
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.3*



REPORT 2/3


# YouTube Video Report

**Generated**: 2025-12-13 01:03:29
**Model**: meta-llama/Llama-3.1-8B-Instruct
**Pipeline Version**: 2.3

---

## Video Information

- **Title**: How well do I know ROBLOX | ALBERTGUESSR
- **Channel**: Flamingo
- **Category (ML Model)**: Entertainment 
- **Category (LLM Validated)**: Gaming
- **Published**: 2025-10-13T03:13:19Z
- **Duration**: PT27M10S seconds
- **Video ID**: `RGbDW-hyxxc`
- **URL**: https://www.youtube.com/watch?v=RGbDW-hyxxc

### Detected Languages

- **Video content**: English
- **Comments**: English (16), German (3), Id (1)
- **Report language**: English

---

## Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | 349,586 |
| Likes | 23,507 |
| Comments | 2,703 |
| Engagement Rate | 7.50% |
| Like Rate | 6.72% |
| Comment Rate | 0.77% |

---

## Video Summary

KEY POINT:
In this entertaining video, AlbertGUESSR plays Roblox GeoGuessr, where he is teleported into various Roblox games to test his knowledge and guessing skills. He shares a list of games, categorized by difficulty levels, for viewers to explore.

DETAILED SUMMARY:
AlbertGUESSR embarks on an exciting adventure through Roblox GeoGuessr, a game that challenges his knowledge of various Roblox games. He is transported into different games, including some that are no longer available on Roblox due to updates, and attempts to guess the correct game based on his surroundings. The games are categorized into four difficulty levels: easy, medium, hard, and impossible modes, providing a variety of challenges for AlbertGUESSR to tackle. Throughout the video, viewers can access links to the games featured, allowing them to explore the games for themselves.

---

## Audience Reaction Summary

**Comments Analyzed**: 20 / 2703 (0.7%)

### ML Model Sentiment Analysis

**Distribution:**
- Positive: 6 (30.0%)
- Neutral: 12 (60.0%)
- Negative: 2 (10.0%)

**Model Confidence (Average):**
- P(Positive): 31.7%
- P(Neutral): 59.2%
- P(Negative): 9.1%

**Interpretation**: Mixed audience sentiment with balanced reactions.

---

### LLM Qualitative Analysis

KEY POINT:
The overall reaction to the video is overwhelmingly positive, with viewers excited to see Kaden return and nostalgic for the content from the past.

DETAILED SUMMARY:
The majority opinion is clear: viewers are thrilled to see Kaden back, with comments like "KADEN IS BACK??!!" and "We love to see Kaden back" showcasing their enthusiasm. Many viewers are also appreciative of the content creator's efforts, with comments like "love when kaden helps out small creators like flamingo <3" indicating a sense of gratitude. Some viewers express nostalgia for the old days, with comments like "I miss all the old collabs Albert would do" and "WE NEED DANI, JANDEL, KADEN, DENIS, TEMPRIST, MILES BACK LIKE THE OLDS DAYYSS" highlighting the longing for past collaborations. However, a minority of viewers express sadness and disappointment, particularly regarding the game's shutdown, with comments like "5:21 It's unfortunate to see that it basically got abandoned after the game was shut down." There is no notable code-switching in the comments.

SENTIMENT BREAKDOWN:
Positive: 85%  Negative: 10%  Neutral: 5%

---

## Top Comments

1. **@@ilikecats382** (2283 likes): Yes we love to see Kaden back...

2. **@@ItsjustmeKayla** (637 likes): love when kaden helps out small creators like flamingo <3...

3. **@@zhyyblox** (322 likes): WE NEED DANI, JANDEL, KADEN, DENIS, TEMPRIST, MILES BACK LIKE THE OLDS DAYYSS...

4. **@@LucieeMotlová** (568 likes): 5:21  It's unfortunate to see that it basically got abandoned after the game was shut down....

5. **@@Loveforsara** (697 likes): 0:10 KADEN IS BACK??!!! OMG OMGG...

---

## Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- Token Efficiency: Dynamic token allocation based on input length
- Quality Gate: Automatic validation and regeneration for quality assurance
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.3*



REPORT 3/3


# YouTube Video Report

**Generated**: 2025-12-13 01:04:00
**Model**: meta-llama/Llama-3.1-8B-Instruct
**Pipeline Version**: 2.3

---

## Video Information

- **Title**: 劇場版『チェンソーマン レゼ篇』オープニングムービー 　主題歌：米津玄師「IRIS OUT」“Chainsaw Man – The Movie: Reze Arc” – Opening Movie
- **Channel**: Kenshi Yonezu  米津玄師
- **Category (ML Model)**: Entertainment 
- **Category (LLM Validated)**: Entertainment
- **Published**: 2025-10-07T10:59:56Z
- **Duration**: PT2M31S seconds
- **Video ID**: `ux3QETpLcPs`
- **URL**: https://www.youtube.com/watch?v=ux3QETpLcPs

### Detected Languages

- **Video content**: Japanese
- **Comments**: Japanese (19), English (1)
- **Report language**: English

---

## Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | 9,096,175 |
| Likes | 416,328 |
| Comments | 14,691 |
| Engagement Rate | 4.74% |
| Like Rate | 4.58% |
| Comment Rate | 0.16% |

---

## Video Summary

KEY POINT:
The video is the opening movie for the Japanese anime film "Chainsaw Man – The Movie: Reze Arc," featuring the theme song "IRIS OUT" by Kenshi Yonezu.

DETAILED SUMMARY:
The video introduces the upcoming anime film, which is an adaptation of the popular manga series "Chainsaw Man" by Tatsuki Fujimoto. The story follows Denji, a young devil hunter, who meets a girl named Reze and becomes involved in a world of blood battles between humans and devils. The film promises to bring an extravagant and thrilling experience, with a blend of love, desire, and action. The video also highlights the voice cast and the production team behind the film, including MAPPA, the studio behind other notable anime series. The trailer showcases the movie's action-packed sequences and hints at the complex relationships between the characters.

---

## Audience Reaction Summary

**Comments Analyzed**: 20 / 14691 (0.1%)

### ML Model Sentiment Analysis

**Distribution:**
- Positive: 15 (75.0%)
- Neutral: 4 (20.0%)
- Negative: 1 (5.0%)

**Model Confidence (Average):**
- P(Positive): 64.5%
- P(Neutral): 30.5%
- P(Negative): 5.1%

**Interpretation**: Highly positive audience reception!

---

### LLM Qualitative Analysis

KEY POINT:
Viewers are overwhelmingly enthusiastic about the opening sequence of the movie, praising its emotional impact, beautiful animation, and the way it sets the tone for the story.

DETAILED SUMMARY:
The majority of viewers (around 90%) are extremely positive about the opening sequence, with many commenting on its emotional impact and how it effectively captures the essence of the story. They appreciate how the sequence seamlessly integrates the opening and ending themes, making the whole experience feel cohesive and engaging. Many viewers are also fond of the character interactions, particularly the relationship between Akira, Denji, and Power, and the way the animation brings out the emotions and personalities of each character. Some viewers (around 5%) have noticed the attention to detail in the animation, such as the way the characters' expressions and movements are designed to be realistic and endearing. A few viewers (around 5%) also appreciate the use of different camera angles and perspectives, such as the bird's-eye view shots.

SENTIMENT BREAKDOWN:
Positive: 90%  Negative: 5%  Neutral: 5%

---

## Top Comments

1. **@@YAMIGMISAMA** (29539 likes): 映画館でこのムービーと一緒に大音量で曲が流れた時の幸福感エグかった...

2. **@@summer-xf2zo** (21043 likes): レゼ編はマジで、OPで「始まるぞ！！」って感じて、EDで「終わった......。」って感じられる最高の作品。...

3. **@@syu-rikka7965** (4349 likes): OPとして独立してるんじゃなくて、その中で確実に話が進んでるし、登場人物全員にしっかりフォーカスを置いてたりしてるのがもう完全に完成しきってるんだって、もう感動です。...

4. **@@harucha6280** (736 likes): ワガママな妹と、世話を焼いてる兄2人みたいな関係性の早川家マジで好き...

5. **@@漢梅トレイン** (18128 likes): 1:05 ここのポチタダンスまじで可愛すぎる...

---

## Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- Token Efficiency: Dynamic token allocation based on input length
- Quality Gate: Automatic validation and regeneration for quality assurance
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.3*


In [ ]:
from pathlib import Path
import re
from datetime import datetime

base_dir = Path("/content/drive/MyDrive/25_sch/ML/reports_batch")
base_dir.mkdir(parents=True, exist_ok=True)

def extract_video_id(report_md: str) -> str:
    m = re.search(r"Video ID.*?`([A-Za-z0-9_\-]+)`", report_md)
    return m.group(1) if m else "unknown"

def extract_title(report_md: str) -> str:
    # looks for: - **Title**: ...
    m = re.search(r"- \*\*Title\*\*: (.+)", report_md)
    if not m:
        return "untitled"
    title = m.group(1).strip()
    title = re.sub(r"[^\w\-]+", "_", title)
    title = re.sub(r"_+", "_", title).strip("_")
    return title[:50] or "untitled"

def save_all_reports(reports, prefix: str = "youtube_report"):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    for i, report_md in enumerate(reports, 1):
        video_id = extract_video_id(report_md)
        title_slug = extract_title(report_md)
        filename = f"{prefix}_{i:02d}_{ts}_{video_id}_{title_slug}.md"
        path = base_dir / filename

        with open(path, "w", encoding="utf-8") as f:
            f.write(report_md)

        print(f"Saved REPORT {i}/{len(reports)} → {path}")

# example
save_all_reports(reports, prefix="yt_report")

## 4. YouTube API Integration (On-The-Fly Application)

Generate reports directly from YouTube URLs.

In [26]:
import re
import isodate
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


class YouTubeAPIClient:
    """YouTube API client for fetching video data"""

    def __init__(self, api_key: str):
        self.youtube = build('youtube', 'v3', developerKey=api_key)

    def extract_video_id(self, url: str) -> Optional[str]:
        """Extract video ID from YouTube URL"""
        patterns = [
            r'(?:v=|\/)([0-9A-Za-z_-]{11}).*',
            r'(?:embed\/|v\/|youtu.be\/)([0-9A-Za-z_-]{11})'
        ]
        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                return match.group(1)
        return None

    def get_video_info(self, video_id: str) -> Optional[Dict]:
        """Fetch video information"""
        try:
            request = self.youtube.videos().list(
                part='snippet,statistics,contentDetails',
                id=video_id
            )
            response = request.execute()

            if not response['items']:
                return None

            item = response['items'][0]
            snippet = item['snippet']
            statistics = item['statistics']
            content_details = item['contentDetails']

            duration = isodate.parse_duration(content_details['duration']).total_seconds()

            return {
                'video_info': {
                    'video_id': video_id,
                    'title': snippet['title'],
                    'description': snippet['description'],
                    'channel_title': snippet['channelTitle'],
                    'published_at': snippet['publishedAt'],
                    'category_id': snippet['categoryId'],
                    'view_count': int(statistics.get('viewCount', 0)),
                    'like_count': int(statistics.get('likeCount', 0)),
                    'comment_count': int(statistics.get('commentCount', 0)),
                    'duration': int(duration)
                },
                'comments': []
            }

        except HttpError as e:
            print(f"API Error: {e}")
            return None

    def get_comments(self, video_id: str, max_results: int = 100) -> List[Dict]:
        """Fetch video comments"""
        try:
            comments = []
            request = self.youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=min(max_results, 100),
                order='relevance'
            )

            while request and len(comments) < max_results:
                response = request.execute()

                for item in response['items']:
                    comment = item['snippet']['topLevelComment']['snippet']
                    comments.append({
                        'author': comment['authorDisplayName'],
                        'text': comment['textDisplay'],
                        'like_count': comment['likeCount'],
                        'published_at': comment['publishedAt']
                    })

                if 'nextPageToken' in response and len(comments) < max_results:
                    request = self.youtube.commentThreads().list(
                        part='snippet',
                        videoId=video_id,
                        pageToken=response['nextPageToken'],
                        maxResults=min(max_results - len(comments), 100),
                        order='relevance'
                    )
                else:
                    break

            return comments

        except HttpError as e:
            print(f"Comments API Error: {e}")
            return []


def generate_report_from_url(video_url: str, api_key: str,
                              config: Optional['PipelineConfig'] = None,
                              model_manager: Optional['ModelManager'] = None,
                              force_reload: bool = False) -> str:
    """
    Generate report from YouTube URL.

    Args:
        video_url: YouTube video URL
        api_key: YouTube API key
        config: Pipeline configuration (None for default)
        model_manager: Existing model manager (None to auto-create)
        force_reload: Force reload model if True

    Returns:
        Generated report string
    """
    print("=" * 80)
    print("GENERATING REPORT FROM URL")
    print("=" * 80)

    if config is None:
        config = PipelineConfig()

    api_client = YouTubeAPIClient(api_key)

    # Extract video ID
    video_id = api_client.extract_video_id(video_url)
    if not video_id:
        raise ValueError("Invalid YouTube URL")

    print(f"Video ID: {video_id}")

    # Fetch video info
    print("  Fetching video information...")
    video_data = api_client.get_video_info(video_id)
    if not video_data:
        raise ValueError("Could not fetch video information")

    print(f"  Video: {video_data['video_info']['title'][:50]}...")

    # Fetch comments
    print(f"  Fetching comments (max: {config.max_comments_to_process})...")
    comments = api_client.get_comments(video_id, config.max_comments_to_process)
    video_data['comments'] = comments
    print(f"  Collected {len(comments)} comments\n")

    # Get or create model manager
    if model_manager is None:
        model_manager = get_or_create_model_manager(config, force_reload)
    else:
        print("Using provided model manager\n")

    # Generate report
    report_gen = ReportGenerator(config, model_manager)
    report = report_gen.generate_full_report(video_data)

    print(f"\n{'=' * 80}")
    print("REPORT COMPLETED")
    print("=" * 80)

    # Print efficiency reports
    print(model_manager.get_token_efficiency_report())
    print(report_gen.quality_gate.get_quality_report())

    return report


print("YouTube API integration loaded")

YouTube API integration loaded


### 4.1 Generate Report from URL

Set your YouTube API key and run to generate a report from any YouTube video.

In [28]:
# Set your YouTube API key (get from https://console.developers.google.com)
YOUR_YOUTUBE_API_KEY = "[REDACTED]"

# Example usage
report = generate_report_from_url(
    "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
    YOUR_YOUTUBE_API_KEY
)
display(Markdown(report))

GENERATING REPORT FROM URL
Video ID: dQw4w9WgXcQ
  Fetching video information...
  Video: Rick Astley - Never Gonna Give You Up (Official Vi...
  Fetching comments (max: 100)...
  Collected 100 comments

Reusing existing model: meta-llama/Llama-3.1-8B-Instruct

Processing: Rick Astley - Never Gonna Give You Up (Official Vi...
  Generating video summary...
    [Input language: English]
      [Input: 675 tokens, Max output: 384]
      [Output: 193 tokens]
    [Output language: English]

  Analyzing sentiment with ML model...
    ML Sentiment: Pos 17.0% | Neu 59.0% | Neg 24.0%
  Generating reaction summary...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


    [Comment languages: {'English': 96, 'Dutch': 2, 'Cy': 1, 'Afrikaans': 1}]
      [Input: 2463 tokens, Max output: 512]
      [Output: 236 tokens]
    [Output language: English]
  Calculating metrics...
  Formatting report...
  ML Model Prediction: Music
  Validating category with LLM...
      [Input: 343 tokens, Max output: 256]
      [Output: 6 tokens]
    Category 10 validated
  LLM Validated: Music
  Done!

REPORT COMPLETED

╔══════════════════════════════════════════════════════════╗
║              TOKEN EFFICIENCY REPORT                     ║
╠══════════════════════════════════════════════════════════╣
║ Total Calls:               12                         ║
║ Total Input Tokens:      9852                         ║
║ Total Output Tokens:     1633                         ║
║ Tokens Saved:            2176                         ║
║ Avg Tokens/Call:        957.1                         ║
║ Efficiency Gain:        18.9%                         ║
╚═════════════════════════════════

# YouTube Video Report

**Generated**: 2025-12-13 01:06:34
**Model**: meta-llama/Llama-3.1-8B-Instruct
**Pipeline Version**: 2.3

---

## Video Information

- **Title**: Rick Astley - Never Gonna Give You Up (Official Video) (4K Remaster)
- **Channel**: Rick Astley
- **Category (ML Model)**: Music 
- **Category (LLM Validated)**: Music
- **Published**: 2009-10-25T06:57:33Z
- **Duration**: 214 seconds
- **Video ID**: `dQw4w9WgXcQ`
- **URL**: https://www.youtube.com/watch?v=dQw4w9WgXcQ

### Detected Languages

- **Video content**: English
- **Comments**: English (96), Dutch (2), Cy (1), Afrikaans (1)
- **Report language**: English

---

## Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | 1,721,716,692 |
| Likes | 18,675,159 |
| Comments | 2,408,242 |
| Engagement Rate | 1.22% |
| Like Rate | 1.08% |
| Comment Rate | 0.14% |

---

## Video Summary

KEY POINT:
This video is the official 4K remastered version of Rick Astley's iconic 1987 hit single "Never Gonna Give You Up," which was a global chart-topper and a defining song of the era.

DETAILED SUMMARY:
The video features the song's memorable lyrics and Rick Astley's catchy vocals, showcasing the timeless romance and devotion theme. Released in July 1987, "Never Gonna Give You Up" was a massive success, topping the charts in 25 countries and winning the Brit Award for Best single in 1988. The song was written and produced by the legendary songwriting and production trio Stock Aitken and Waterman for Rick Astley's debut album "Whenever You Need Somebody," which went on to sell over 15 million copies worldwide. The music video, directed by Simon West, has become a classic of the era and has surpassed 1 billion views on YouTube since its upload.

---

## Audience Reaction Summary

**Comments Analyzed**: 100 / 2408242 (0.0%)

### ML Model Sentiment Analysis

**Distribution:**
- Positive: 17 (17.0%)
- Neutral: 59 (59.0%)
- Negative: 24 (24.0%)

**Model Confidence (Average):**
- P(Positive): 17.7%
- P(Neutral): 57.0%
- P(Negative): 25.3%

**Interpretation**: Mixed audience sentiment with balanced reactions.

---

### LLM Qualitative Analysis

KEY POINT:
The audience reaction to the YouTube video is overwhelmingly positive, with viewers expressing their love for the song "Never Gonna Give You Up" by Rick Astley, as well as their amusement and nostalgia for the "Rickrolling" meme.

DETAILED SUMMARY:
The majority of viewers are in high spirits, sharing their experiences of being "rickrolled" and joking about the meme's persistence and widespread impact. Many commenters express their admiration for the song, with some even suggesting it as the national anthem of the internet. Some viewers share humorous anecdotes about getting rickrolled in various situations, such as school, work, or even by AI-generated content. Notable minority views include a few commenters who are less enthusiastic, either because they didn't understand the joke or because they felt tricked. A small number of commenters also express a mix of nostalgia and melancholy, reminiscing about the good old days of the internet. Language patterns show a mix of informal and playful language, with some commenters using emojis and humorously exaggerated expressions.

SENTIMENT BREAKDOWN:
Positive: 85%  Negative: 5%  Neutral: 10%

---

## Top Comments

1. **@@YouTube** (146373 likes): can confirm: he never gave us up...

2. **@@JetpackGamer456** (417474 likes): Ok let’s be honest here<br>Rick rolling is the one meme that will never die...

3. **@@SilentMemer** (124554 likes): Nobody can fool me anymore with this, I now remember every letter and number of the link...

4. **@@CinematicCaptures** (353478 likes): I didn&#39;t get rickrolled today, I just really enjoy this song...

5. **@@MrC37** (5 likes): I don&#39;t care, this song SLAPS...

---

## Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- Token Efficiency: Dynamic token allocation based on input length
- Quality Gate: Automatic validation and regeneration for quality assurance
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.3*


In [ ]:
from pathlib import Path
import re
from datetime import datetime

# Base directory for saving reports
base_dir = Path("/content/drive/MyDrive/25_sch/ML/reports")
base_dir.mkdir(parents=True, exist_ok=True)

def slugify(text: str, max_len: int = 50) -> str:
    text = re.sub(r"[^\w\-]+", "_", text)
    text = re.sub(r"_+", "_", text).strip("_")
    return text[:max_len] or "report"

def extract_video_id(report_md: str) -> str:
    # looks for a line like: - **Video ID**: `EmeW6li6bbo`
    m = re.search(r"Video ID.*?`([A-Za-z0-9_\-]+)`", report_md)
    return m.group(1) if m else "unknown"

def save_single_report(report_md: str, prefix: str = "report"):
    video_id = extract_video_id(report_md)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{prefix}_{ts}_{video_id}.md"
    path = base_dir / filename

    with open(path, "w", encoding="utf-8") as f:
        f.write(report_md)

    print(f"Saved report to: {path}")

# example
save_single_report(report, prefix="youtube_report")


## 5. Cleanup (Optional)

In [ ]:
# Release model from memory when done
# cleanup_model()

## 6. Pipeline Summary

### Features

**Token Efficiency:**
- Dynamic token allocation based on input length
- Three modes: conservative, adaptive (default), aggressive
- Typical 20-40% token savings

**Quality Gate:**
- Automatic validation of generated summaries
- 5 quality checks: length, diversity, relevance, structure, error detection
- Auto-regeneration with conservative temperature on failure

**Model Integration:**
- Category prediction (BERT + TF-IDF)
- LLM category validation
- Sentiment analysis (BERT)

**Report Content:**
- Video information and metadata
- Engagement metrics
- Two-part summary (KEY POINT + DETAILED SUMMARY)
- ML sentiment analysis results
- LLM qualitative analysis
- Top comments

### Configuration Reference

```python
# Token efficiency
config.enable_dynamic_tokens = True
config.token_efficiency_mode = "adaptive"  # conservative/adaptive/aggressive

# Quality gate
config.enable_quality_gate = True
config.min_summary_length = 100
config.min_keyword_diversity = 10
config.max_regeneration_attempts = 2

# Model integration
config.use_category_model = True
config.use_sentiment_model = True
config.enable_llm_category_correction = True
```